In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [2]:
# 수동 입력 데이터 프레임
minimum_linkid_mapping_df = pd.read_excel('./영등포_link_mapping_check_df.xlsx')
minimum_linkid_mapping_df.loc[minimum_linkid_mapping_df['max_spd'] == 10, 'max_spd'] = 30 # max_spd 이상치 실제 반영
minimum_linkid_mapping_df.to_csv('./영등포_link_mapping_check_df_modify.csv', index=False, encoding='utf-8-sig')
# 샘플 교통현황 데이터
traffic_realtime_df = pd.read_csv('./its_traffic_241001_02/its_traffic_data_2024-10-02_182716.csv', encoding='utf-8-sig')
minimum_linkid_mapping_df_cp = minimum_linkid_mapping_df[['주차장 이름', 'link_id', 'max_spd']].copy()

In [3]:
def add_speed(row):
    link_id = row['link_id']
    spd = traffic_realtime_df[traffic_realtime_df['linkId'] == int(link_id)]['speed']
    
    return spd.values[0]

def cal_confusion(df):
    parking_lot_list = df['주차장 이름'].unique()
    for parking_lot in parking_lot_list:
        df.loc[df['주차장 이름'] == parking_lot, 'confusion'] = df['speed'] / df['max_spd']
    
    # confusion 열의 평균을 주차장 이름 별로 계산
    confusion_df = df.groupby('주차장 이름')['confusion'].mean().reset_index()
    confusion_df.loc[confusion_df['confusion'] >= 1, 'confusion'] = 1.00
    confusion_df['confusion'] = 1 - confusion_df['confusion']
    confusion_df['confusion'] = confusion_df['confusion'].round(2)
    
    return confusion_df

minimum_linkid_mapping_df_cp['speed'] = minimum_linkid_mapping_df_cp.progress_apply(add_speed, axis=1)
cal_confusion_df = cal_confusion(minimum_linkid_mapping_df_cp)

100%|██████████| 226/226 [00:00<00:00, 2700.04it/s]


In [4]:
cal_confusion_df.to_csv('./result_traffic_info.csv', index=False, encoding='utf-8-sig')

In [5]:
cal_confusion_df

,주차장 이름,confusion
0,63스퀘어(63빌딩),0.58
1,IFC 서울 주차장,0.61
2,금융투자협회,0.81
3,당산2동(구),0.52
4,당산근린공원 공영(구),0.63
5,당산노외 공영주차장(시),0.73
6,대림1동공영(구),0.60
7,대림운동장(구),0.49
8,도림동 공영주차장(구),0.38
9,도림신협본점 옥외주차장,0.29
